<a href="https://colab.research.google.com/github/djangointhehills/Perceptron/blob/main/Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import numpy as np
import math
import copy
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron as sklearn_Perceptron
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix

# Activations

In [2]:
def bad(x):
    return 1

In [3]:
def step_function(x):
    if x < 0:
      return -1
    else:
      return 1

In [4]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

# Perceptron

In [5]:
class Perceptron_Unit:
    def __init__(self, learning_rate, activation_f, max_iterations, weights):
        self.weights = [0] * weights
        self.activation_f = activation_f
        self.learning_rate = learning_rate
        self.max_iterations = max_iterations

    def update_weights(self, x, y_hat, y):
        for i in range(len(self.weights)):
            self.weights[i] += self.learning_rate * (y - y_hat) * x[i]

    def update(self, x,y):
        pred = self.activation_f(np.dot(self.weights, x))
        if pred != y:
          self.update_weights(x, pred, y)

    def predict(self, x):
        prediction = self.activation_f(np.dot(self.weights, x))
        return prediction
    
    def get_weights(self):
      return self.weights

In [6]:
class Perceptron:
    def __init__(self, learning_rate, activation_f, max_iterations):
        self.perceptrons = {}
        self.activation_f = activation_f
        self.learning_rate = learning_rate
        self.max_iterations = max_iterations

    def train(self, X, Y):
        # Get the number of weights needed.
        num_weights = 1 + len(X[0])

        # Create perceptron for each label.
        labels = np.unique(Y)
        if len(labels) < 3:
          self.perceptrons[labels[0]] = Perceptron_Unit(self.learning_rate, self.activation_f, self.max_iterations, num_weights)
        else:
          for label in labels:
            self.perceptrons[label] = Perceptron_Unit(self.learning_rate, self.activation_f, self.max_iterations, num_weights)

        epoch = 0
        train = True
        while train and epoch < self.max_iterations:
          epoch += 1
          train = False
          for i in range(len(X)):
            # Add bias term
            instance = np.insert(X[i], 0, 1)

            prediction = (-math.inf, None)

            # Get max probability label.
            for label in self.perceptrons: 
              prob = self.perceptrons[label].predict(instance)
              if Y[i] == label:
                  if prob > prediction[0]:
                    prediction = (prob, label)
              else:
                if prob > prediction[0]:
                  prediction = (prob, label)
      
            # Check if prediction was correct. If not, update.
            if prediction[1] != Y[i]:
              train = True
              for label in self.perceptrons:
                if Y[i] == label:
                  self.perceptrons[label].update(instance, 1)
                else:
                  self.perceptrons[label].update(instance, 0)

    def predict(self, X):
        predictions = []

        for i in range(len(X)):
          # Add bias term
          instance = np.insert(X[i], 0, 1)

          prediction = (-math.inf, None)

          # Get max probability label.
          for label in self.perceptrons: 
            prob = self.perceptrons[label].predict(instance)
            if prob > prediction[0]:
              prediction = (prob, label)

          predictions.append(prediction[1])
        
        return predictions

    def predict_score(self, X,Y):
        predictions = self.predict(X)
        accuracy = self.get_acuracy(predictions,Y)
        print("Accuracy =", accuracy)

    def get_acuracy(self, preds,y):
        no_correct = 0
        for i in range(len(preds)):
          if preds[i] == y[i]:
            no_correct += 1
        accuracy = no_correct / len(preds)
        return accuracy

    def get_weights(self,):
        weights = {}
        for unit in self.perceptrons:
          weights[unit] = self.perceptrons[unit].get_weights()
        return weights

# Experiementation with Iris Data Set

# Useful Code

In [7]:
def show_confusion(pred, y_test):
    cm=metrics.confusion_matrix(y_test,pred)
    print(cm)
    plt.imshow(cm, cmap='binary')

## Load Data

In [8]:
iris = datasets.load_iris()

In [9]:
x_data = iris['data']
y_data = iris['target']

In [10]:
x_data.shape

(150, 4)

In [46]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.33)

# Compare with Sklearn Percetron

## SKLearn

In [47]:
sk_model = sklearn_Perceptron()
sk_model.fit(X_train, y_train)
sk_model.score(X_test, y_test)

0.7

## My Model

In [48]:
next_model = Perceptron(.1, step_function, 100)
next_model.train(X_train, y_train)
next_model.predict_score(X_test, y_test)

Accuracy = 0.8
